### Extraction des moments de tchebichef des lésion en 2D (01 -> 14)

In [ ]:
from pathlib import Path
import SimpleITK as sitk
import numpy as np
import csv

data_path = Path.cwd().parent.parent / "data"

tchbichef_2d_dir_path = data_path / "14_2D_momentum_extraction_csv_dir"
if not tchbichef_2d_dir_path.exists():
    tchbichef_2d_dir_path.mkdir(parents = True, exist_ok = True)

# A CHOISIR:
ORDRE_CIBLE = 10 # 10 * 10 = 100 coefficients

In [13]:
def tchebichef_polynomials(N, order):
    M = np.zeros((order, N))
    x = np.arange(N)
    M[0, :] = 1.0
    if order > 1: M[1, :] = (2.0 * x - N + 1.0) / N
    for p in range(2, order):
        M[p, :] = ((2.0*p-1.0)*(2.0*x-N+1.0)/(p*N))*M[p-1,:] - ((p-1.0)/p)*(1.0-((p-1.0)**2/(N**2)))*M[p-2,:]
    return M

def calculate_rho(N, order):
    rho = np.zeros(order); rho[0] = N
    for p in range(1, order): rho[p] = (N**2-p**2)/(N**2) * (2*p-1)/(2*p+1) * rho[p-1]
    return rho

def decompose_2d(image, requested_order):
    H, W = image.shape
    actual_order_h, actual_order_w = min(requested_order, H), min(requested_order, W)
    M_h = tchebichef_polynomials(H, actual_order_h)
    M_w = tchebichef_polynomials(W, actual_order_w)
    rho_h, rho_w = calculate_rho(H, actual_order_h), calculate_rho(W, actual_order_w)
    T_raw = M_h @ image @ M_w.T
    C_pq = 1.0 / np.outer(rho_h, rho_w)
    T_pq_small = T_raw * C_pq
    T_final = np.zeros((requested_order, requested_order))
    T_final[:actual_order_h, :actual_order_w] = T_pq_small
    return np.round(T_final, 3) # Arrondi à 3 décimales dès le calcul

def reconstruct_2d(moments, shape_originale):
    H, W = shape_originale
    order = moments.shape[0]
    M_h = tchebichef_polynomials(H, order)
    M_w = tchebichef_polynomials(W, order)
    return M_h.T @ moments @ M_w

In [ ]:
"""
01 -> 14
Ce script prend environ 2 minutes
Extraction des moments 2D d'images médicales
"""

mslesseg_path = data_path / "01_MSLesSeg_Dataset"

# Parcours de tous les patients et leurs sous-dossiers
for mask_path in mslesseg_path.rglob("*_MASK.nii.gz"):
    arg = mask_path.stem.split("_")
    if len(arg) == 3:
        patient_id, timepoint = arg[0], arg[1]
    elif len(arg) == 2:
        patient_id, timepoint = arg[0], None
    patient_dir = mask_path.parent

    # Chargement du masque et identification des lésions
    mask_sitk = sitk.ReadImage(mask_path)
    mask_bin = sitk.Cast(mask_sitk > 0, sitk.sitkUInt8)
    lesion_cc = sitk.RelabelComponent(sitk.ConnectedComponent(mask_bin))  # L1 = plus grosse
    mask_arr = sitk.GetArrayFromImage(lesion_cc)
    num_lesions = int(mask_arr.max())

    t1_path = list(patient_dir.glob("*_T1.nii.gz"))[0]
    t2_path = list(patient_dir.glob("*_T2.nii.gz"))[0]
    flair_path = list(patient_dir.glob("*_FLAIR.nii.gz"))[0]

    for modality_path in [t1_path, t2_path, flair_path]:
        modality_name = (modality_path.stem.split("_")[-1]).split(".")[0]
        img_arr = sitk.GetArrayFromImage(sitk.ReadImage(modality_path))

        print(f"Processing: {patient_id} {timepoint} {modality_name}: {num_lesions} lésions")

        for l_id in range(1, num_lesions + 1):
            lesion_mask_3d = (mask_arr == l_id)
            areas = lesion_mask_3d.sum(axis=(1, 2))
            best_z = np.argmax(areas)

            # Extraction ROI
            slice_mask = lesion_mask_3d[best_z, :, :]
            ys, xs = np.where(slice_mask)
            if len(ys) < 2 or len(xs) < 2:
                continue

            roi = img_arr[best_z, ys.min():ys.max()+1, xs.min():xs.max()+1] * slice_mask[ys.min():ys.max()+1, xs.min():xs.max()+1]

            # Normalisation et décomposition
            max_v = np.max(roi)
            roi_norm = roi / max_v if max_v > 0 else roi

            moments = decompose_2d(roi_norm, ORDRE_CIBLE)

            # Sauvegarde CSV
            if len(arg) == 3:
                csv_name = f"{patient_id}_{timepoint}_{modality_name}_L{l_id}_Cheby.csv"
            elif len(arg) == 2:
                csv_name = f"{patient_id}_{modality_name}_L{l_id}_Cheby.csv"
            csv_file = tchbichef_2d_dir_path / csv_name
            
            with csv_file.open("w", newline="") as f:
                csv.writer(f).writerows(moments)

print("\n[SUCCÈS] Toutes les lésions de tous les patients ont été extraites.")


Processing: P54 None T1: 5 lésions
Processing: P54 None T2: 5 lésions
Processing: P54 None FLAIR: 5 lésions
Processing: P55 None T1: 37 lésions
Processing: P55 None T2: 37 lésions
Processing: P55 None FLAIR: 37 lésions
Processing: P56 None T1: 4 lésions
Processing: P56 None T2: 4 lésions
Processing: P56 None FLAIR: 4 lésions
Processing: P57 None T1: 113 lésions
Processing: P57 None T2: 113 lésions
Processing: P57 None FLAIR: 113 lésions
Processing: P58 None T1: 43 lésions
Processing: P58 None T2: 43 lésions
Processing: P58 None FLAIR: 43 lésions
Processing: P59 None T1: 15 lésions
Processing: P59 None T2: 15 lésions
Processing: P59 None FLAIR: 15 lésions
Processing: P60 None T1: 26 lésions
Processing: P60 None T2: 26 lésions
Processing: P60 None FLAIR: 26 lésions
Processing: P61 None T1: 13 lésions
Processing: P61 None T2: 13 lésions
Processing: P61 None FLAIR: 13 lésions
Processing: P62 None T1: 52 lésions
Processing: P62 None T2: 52 lésions
Processing: P62 None FLAIR: 52 lésions
Proc